# <font color=green> Sentiment Analysis in Russian Drama </font>

Practice using the Russian Drama corpus with the **Dostoevsky** and **polyglot** libraries

1. Get Data
    - Access the drama corpus API
    - Normalize json data
    - Get play text
    - Save data as .csv file to folder for easy access
2. Dostoevsky and polyglot libraries for sentiment analysis
    - Clean play data
    - Lemmatize text
    - Do sentiment analysis (get polarity scores) with the polyglot and dostoevsky libraries
    - Do some examination of polarity scores for different authors

Import necessary libraries.


In [3]:
import requests
import pandas as pd
import numpy as np
import json
import string
import re

from pandas.io.json import json_normalize
from nltk.corpus import stopwords 
from pymystem3 import Mystem
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from polyglot.text import Text


Using TensorFlow backend.


## 1. Get Data
***

Access API and inspect.

In [4]:
url = 'http://dracor.org/api/corpora/rus'
r = requests.get(url)
json_data = r.json()

In [5]:
print (json_data)

{'name': 'rus', 'title': 'Russian Drama Corpus', 'repository': 'https://github.com/dracor-org/rusdracor', 'dramas': [{'id': 'rus000138', 'name': 'andreyev-ne-ubiy', 'title': 'Не убий', 'subtitle': '–', 'author': {'key': 'wikidata:Q310866', 'name': 'Андреев, Леонид Николаевич'}, 'authors': [{'key': 'wikidata:Q310866', 'name': 'Андреев, Леонид Николаевич'}], 'yearNormalized': '1913', 'source': 'Библиотека Максима Мошкова (lib.ru)', 'sourceUrl': 'http://az.lib.ru/a/andreew_l_n/text_1330.shtml', 'printYear': '1913', 'premiereYear': '1913', 'writtenYear': '1913', 'networkSize': '23', 'networkdataCsvUrl': 'https://dracor.org/api/corpora/rus/play/andreyev-ne-ubiy/networkdata/csv', 'wikidataId': 'Q59357856'}, {'id': 'rus000137', 'name': 'andreyev-mysl', 'title': 'Мысль', 'subtitle': 'Современная трагедия в трех действиях и шести картинах', 'author': {'key': 'wikidata:Q310866', 'name': 'Андреев, Леонид Николаевич'}, 'authors': [{'key': 'wikidata:Q310866', 'name': 'Андреев, Леонид Николаевич'}],

Normalize json data.

In [6]:
df = pd.io.json.json_normalize(json_data)

df.head()

,dramas,name,repository,title
0,"[{'id': 'rus000138', 'name': 'andreyev-ne-ubiy...",rus,https://github.com/dracor-org/rusdracor,Russian Drama Corpus


In [7]:
dramadata = df.iloc[0,0]


In [8]:
normalized_data = json_normalize(dramadata)


In [9]:
pd.set_option('display.width', 1000)


In [10]:
normalized_data.head()

,author.key,author.name,authors,id,name,networkSize,networkdataCsvUrl,premiereYear,printYear,source,sourceUrl,subtitle,title,wikidataId,writtenYear,yearNormalized
0,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000138,andreyev-ne-ubiy,23,https://dracor.org/api/corpora/rus/play/andrey...,1913,1913,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1330.shtml,–,Не убий,Q59357856,1913,1913
1,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000137,andreyev-mysl,15,https://dracor.org/api/corpora/rus/play/andrey...,1914,1914,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1290.shtml,Современная трагедия в трех действиях и шести ...,Мысль,Q59355429,1913,1914
2,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000194,andreyev-k-zvezdam,13,https://dracor.org/api/corpora/rus/play/andrey...,1906,None,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1010.shtml,NaN,К звёздам,Q19172474,1905,1906
3,wikidata:Q729569,"Афиногенов, Александр Николаевич","[{'key': 'wikidata:Q729569', 'name': 'Афиноген...",rus000167,afinogenov-mashenka,12,https://dracor.org/api/corpora/rus/play/afinog...,1941,1941,Wikisource,https://ru.wikisource.org/wiki/Машенька_(Афино...,Пьеса в 3-х актах 7-ми сценах,Машенька,Q25394256,1940,1941
4,wikidata:Q49080,"Бабель, Исаак Эммануилович","[{'key': 'wikidata:Q49080', 'name': 'Бабель, И...",rus000119,babel-marija,22,https://dracor.org/api/corpora/rus/play/babel-...,1964,1935,Wikilivres/Bibliowiki,https://wikilivres.org/wiki/Мария_(Бабель),Пьеса в 8 картинах,Мария,Q6760975,1934,1935


The above dataframe has all the metadata, but we still need the **text** of each play itself to do any NLP work.  Let's add a column to the dataframe with the spoken word text. 

In [11]:
def get_play_text(name_of_play):
    name = str(name_of_play)
    url = 'http://dracor.org/api/corpora/rus/play/' + name + '/spoken-text'
    r = requests.get(url).text
    return r

Note: applying the above function in the line below takes a bit of time..

In [12]:
normalized_data['play_text'] = normalized_data['name'].apply(get_play_text)

In [13]:
normalized_data.head()

,author.key,author.name,authors,id,name,networkSize,networkdataCsvUrl,premiereYear,printYear,source,sourceUrl,subtitle,title,wikidataId,writtenYear,yearNormalized,play_text
0,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000138,andreyev-ne-ubiy,23,https://dracor.org/api/corpora/rus/play/andrey...,1913,1913,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1330.shtml,–,Не убий,Q59357856,1913,1913,"Что задумались, Василиса Петровна? Я пришел.\n..."
1,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000137,andreyev-mysl,15,https://dracor.org/api/corpora/rus/play/andrey...,1914,1914,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1290.shtml,Современная трагедия в трех действиях и шести ...,Мысль,Q59355429,1913,1914,Он спит.\nДа. Так он спит теперь по целым дням...
2,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000194,andreyev-k-zvezdam,13,https://dracor.org/api/corpora/rus/play/andrey...,1906,None,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1010.shtml,NaN,К звёздам,Q19172474,1905,1906,"Звонит, звонит, а все без толку. За четыре дня..."
3,wikidata:Q729569,"Афиногенов, Александр Николаевич","[{'key': 'wikidata:Q729569', 'name': 'Афиноген...",rus000167,afinogenov-mashenka,12,https://dracor.org/api/corpora/rus/play/afinog...,1941,1941,Wikisource,https://ru.wikisource.org/wiki/Машенька_(Афино...,Пьеса в 3-х актах 7-ми сценах,Машенька,Q25394256,1940,1941,"Кто спрашивает? Его дома нет. И когда будет, н..."
4,wikidata:Q49080,"Бабель, Исаак Эммануилович","[{'key': 'wikidata:Q49080', 'name': 'Бабель, И...",rus000119,babel-marija,22,https://dracor.org/api/corpora/rus/play/babel-...,1964,1935,Wikilivres/Bibliowiki,https://wikilivres.org/wiki/Мария_(Бабель),Пьеса в 8 картинах,Мария,Q6760975,1934,1935,Дорогу всю расшлёпали... Зандберг был на Выриц...


Data is now ready!  The next line saves the above dataframe as a .csv for easy access, and so that the above steps don't need to be repeated.

In [14]:
normalized_data.to_csv('russian_play_data.csv', index=False)

## Sentiment Analysis with the Dostoevsky and polyglot libraries.

***

In [4]:
russian_play_data = pd.read_csv('russian_play_data.csv')


In [5]:
russian_play_data.head()

,author.key,author.name,authors,id,name,networkSize,networkdataCsvUrl,premiereYear,printYear,source,sourceUrl,subtitle,title,wikidataId,writtenYear,yearNormalized,play_text
0,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000138,andreyev-ne-ubiy,23,https://dracor.org/api/corpora/rus/play/andrey...,1913.0,1913.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1330.shtml,–,Не убий,Q59357856,1913.0,1913,"Что задумались, Василиса Петровна? Я пришел.\n..."
1,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000137,andreyev-mysl,15,https://dracor.org/api/corpora/rus/play/andrey...,1914.0,1914.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1290.shtml,Современная трагедия в трех действиях и шести ...,Мысль,Q59355429,1913.0,1914,Он спит.\nДа. Так он спит теперь по целым дням...
2,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000194,andreyev-k-zvezdam,13,https://dracor.org/api/corpora/rus/play/andrey...,1906.0,NaN,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1010.shtml,NaN,К звёздам,Q19172474,1905.0,1906,"Звонит, звонит, а все без толку. За четыре дня..."
3,wikidata:Q729569,"Афиногенов, Александр Николаевич","[{'key': 'wikidata:Q729569', 'name': 'Афиноген...",rus000167,afinogenov-mashenka,12,https://dracor.org/api/corpora/rus/play/afinog...,1941.0,1941.0,Wikisource,https://ru.wikisource.org/wiki/Машенька_(Афино...,Пьеса в 3-х актах 7-ми сценах,Машенька,Q25394256,1940.0,1941,"Кто спрашивает? Его дома нет. И когда будет, н..."
4,wikidata:Q49080,"Бабель, Исаак Эммануилович","[{'key': 'wikidata:Q49080', 'name': 'Бабель, И...",rus000119,babel-marija,22,https://dracor.org/api/corpora/rus/play/babel-...,1964.0,1935.0,Wikilivres/Bibliowiki,https://wikilivres.org/wiki/Мария_(Бабель),Пьеса в 8 картинах,Мария,Q6760975,1934.0,1935,Дорогу всю расшлёпали... Зандберг был на Выриц...


Clean and lemmatize the play text.

In [6]:
stop = list(stopwords.words('russian'))
stop.extend(['это', 'свой', 'то', ' ', 'то', 'который'])
mystem = Mystem() 
exclude = list(string.punctuation)
exclude.extend

def clean_and_lemmatize(text):
    tokens = text.split(" ")
    tokens = [i.strip().lower() for i in tokens]
    tokens = [i for i in tokens if i not in exclude]
    tokens = [re.sub(r'[!|,|\n|?|.|:|;]|\.{1,}$\)', "", item) for item in tokens]
    tokens = [i for i in tokens if i not in stop]
    cleanedtext = " ".join(tokens)
    lemmatized = mystem.lemmatize(cleanedtext.lower())
    lemmatized = [i for i in lemmatized if i not in exclude]
    lemmatized = [i for i in lemmatized if i not in stop]
    lemmatized = " ".join(str(x) for x in lemmatized)
    return lemmatized

russian_play_data["cleaned_play_text"] = russian_play_data["play_text"].apply(clean_and_lemmatize)

russian_play_data.head()

,author.key,author.name,authors,id,name,networkSize,networkdataCsvUrl,premiereYear,printYear,source,sourceUrl,subtitle,title,wikidataId,writtenYear,yearNormalized,play_text,cleaned_play_text
0,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000138,andreyev-ne-ubiy,23,https://dracor.org/api/corpora/rus/play/andrey...,1913.0,1913.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1330.shtml,–,Не убий,Q59357856,1913.0,1913,"Что задумались, Василиса Петровна? Я пришел.\n...",задумываться василиса петровна пришелд думать ...
1,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000137,andreyev-mysl,15,https://dracor.org/api/corpora/rus/play/andrey...,1914.0,1914.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1290.shtml,Современная трагедия в трех действиях и шести ...,Мысль,Q59355429,1913.0,1914,Он спит.\nДа. Так он спит теперь по целым дням...,спитда спать целый день третий орангутанг умир...
2,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000194,andreyev-k-zvezdam,13,https://dracor.org/api/corpora/rus/play/andrey...,1906.0,NaN,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1010.shtml,NaN,К звёздам,Q19172474,1905.0,1906,"Звонит, звонит, а все без толку. За четыре дня...",звонить звонить толк четыре день приходить сид...
3,wikidata:Q729569,"Афиногенов, Александр Николаевич","[{'key': 'wikidata:Q729569', 'name': 'Афиноген...",rus000167,afinogenov-mashenka,12,https://dracor.org/api/corpora/rus/play/afinog...,1941.0,1941.0,Wikisource,https://ru.wikisource.org/wiki/Машенька_(Афино...,Пьеса в 3-х актах 7-ми сценах,Машенька,Q25394256,1940.0,1941,"Кто спрашивает? Его дома нет. И когда будет, н...",спрашивать дома неизвестно ладно передавать пя...
4,wikidata:Q49080,"Бабель, Исаак Эммануилович","[{'key': 'wikidata:Q49080', 'name': 'Бабель, И...",rus000119,babel-marija,22,https://dracor.org/api/corpora/rus/play/babel-...,1964.0,1935.0,Wikilivres/Bibliowiki,https://wikilivres.org/wiki/Мария_(Бабель),Пьеса в 8 картинах,Мария,Q6760975,1934.0,1935,Дорогу всю расшлёпали... Зандберг был на Выриц...,дорога расшлепать зандберг вырица человек жить...


The play text has been cleaned and lemmatized (scroll to the right and note the two far right columns in the rows above). Now it is ready for sentiment analysis.

Let's save this dataframe as a new .csv.

In [7]:
russian_play_data.to_csv('russian_play_data.csv', index=False)

1. Sentiment analysis (polarity score) with polyglot.

In [8]:
russian_play_data = pd.read_csv('russian_play_data.csv')


In [14]:

def polglot_sentiment_analysis(doc):
    text = Text(doc)
    text.language = 'ru'
    polarity_score = text.polarity
    return polarity_score

russian_play_data['polyglot_polarity_score'] = russian_play_data["cleaned_play_text"] .apply(polglot_sentiment_analysis)

In [10]:
russian_play_data.head()

,author.key,author.name,authors,id,name,networkSize,networkdataCsvUrl,premiereYear,printYear,source,sourceUrl,subtitle,title,wikidataId,writtenYear,yearNormalized,play_text,cleaned_play_text,polyglot_polarity_score
0,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000138,andreyev-ne-ubiy,23,https://dracor.org/api/corpora/rus/play/andrey...,1913.0,1913.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1330.shtml,–,Не убий,Q59357856,1913.0,1913,"Что задумались, Василиса Петровна? Я пришел.\n...",задумываться василиса петровна пришелд думать ...,0.182137
1,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000137,andreyev-mysl,15,https://dracor.org/api/corpora/rus/play/andrey...,1914.0,1914.0,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1290.shtml,Современная трагедия в трех действиях и шести ...,Мысль,Q59355429,1913.0,1914,Он спит.\nДа. Так он спит теперь по целым дням...,спитда спать целый день третий орангутанг умир...,0.033730
2,wikidata:Q310866,"Андреев, Леонид Николаевич","[{'key': 'wikidata:Q310866', 'name': 'Андреев,...",rus000194,andreyev-k-zvezdam,13,https://dracor.org/api/corpora/rus/play/andrey...,1906.0,NaN,Библиотека Максима Мошкова (lib.ru),http://az.lib.ru/a/andreew_l_n/text_1010.shtml,NaN,К звёздам,Q19172474,1905.0,1906,"Звонит, звонит, а все без толку. За четыре дня...",звонить звонить толк четыре день приходить сид...,0.024390
3,wikidata:Q729569,"Афиногенов, Александр Николаевич","[{'key': 'wikidata:Q729569', 'name': 'Афиноген...",rus000167,afinogenov-mashenka,12,https://dracor.org/api/corpora/rus/play/afinog...,1941.0,1941.0,Wikisource,https://ru.wikisource.org/wiki/Машенька_(Афино...,Пьеса в 3-х актах 7-ми сценах,Машенька,Q25394256,1940.0,1941,"Кто спрашивает? Его дома нет. И когда будет, н...",спрашивать дома неизвестно ладно передавать пя...,0.268482
4,wikidata:Q49080,"Бабель, Исаак Эммануилович","[{'key': 'wikidata:Q49080', 'name': 'Бабель, И...",rus000119,babel-marija,22,https://dracor.org/api/corpora/rus/play/babel-...,1964.0,1935.0,Wikilivres/Bibliowiki,https://wikilivres.org/wiki/Мария_(Бабель),Пьеса в 8 картинах,Мария,Q6760975,1934.0,1935,Дорогу всю расшлёпали... Зандберг был на Выриц...,дорога расшлепать зандберг вырица человек жить...,0.158491


A new column has been made which contains the polarity score from polyglot for each play. 

Let's figure out what the average polarity score is for the works of each writer in the corpus.

In [34]:
russian_play_data.groupby('author.name', as_index=False)['polyglot_polarity_score'].mean().sort_values(by=['polyglot_polarity_score'])

,author.name,polyglot_polarity_score
6,"Глинка, Фёдор Николаевич",-0.155397
25,"Майков, Василий Иванович",-0.040612
14,"Капнист, Василий Васильевич",-0.006171
46,"Толстой, Алексей Константинович",0.020643
32,"Озеров, Владислав Александрович",0.024096
15,"Кедрин, Дмитрий Борисович",0.028571
24,"Ломоносов, Михаил Васильевич",0.041109
43,"Судовщиков, Николай Родионович",0.048093
51,"Херасков, Михаил Матвеевич",0.050291
3,"Бельский, Владимир Иванович",0.059305


Fedor Glinka's work has the lowest polarity scores with polyglot, and Ekaterina Dashkova's has the highest.

## <font color=red>2. Sentiment analysis (polarity score) with Dostoevsky. THIS PART NEEDS WORK </font>

In [11]:
tokenizer = RegexTokenizer()


In [13]:
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [29]:

def dostoevsky_sentiment_analysis(doc):
    tokens = doc.split(" ")
    #print (tokens)
    results = model.predict(tokens, k=2)
  #  print (results)
    print (len(results))
    tokenlength = len(results)
    polaritysum = 0
    for i in results:
        print (i)
    #for token, sentiment in zip(tokens, results):
        #print(tokens, '->', sentiment)
    return

russian_play_data['dostoevsky_polarity_score'] = russian_play_data["cleaned_play_text"].apply(dostoevsky_sentiment_analysis)

8426
{'neutral': 0.9879562258720398, 'skip': 0.061885979026556015}
{'neutral': 1.0000100135803223, 'skip': 0.0013770253863185644}
{'neutral': 0.982567548751831, 'skip': 0.08270734548568726}
{'neutral': 1.0000100135803223, 'skip': 0.0006978511810302734}
{'neutral': 1.0000100135803223, 'negative': 0.016924913972616196}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.9814634323120117, 'skip': 0.04604391008615494}
{'neutral': 1.0000100135803223, 'negative': 0.016924913972616196}
{'neutral': 1.0000100135803223, 'skip': 0.005230126902461052}
{'neutral': 0.9850529432296753, 'skip': 0.9416641592979431}
{'neutral': 0.9433575868606567, 'negative': 0.016924913972616196}
{'neutral': 0.9679093360900879, 'negative': 0.020974241197109222}
{'neutral': 0.9899964332580566, 'positive': 0.23935934901237488}
{'neutral': 0.9525841474533081, 'skip': 0.00574024673551321}
{'neutral': 0.9626831412315369, 'skip': 0.013646835461258888}
{'neutral': 1.0000100135803223, 'skip': 0.00

{'neutral': 0.996837317943573, 'skip': 0.04743587225675583}
{'neutral': 1.0000100135803223, 'skip': 0.004341503139585257}
{'neutral': 1.0000100135803223, 'skip': 0.0003669706347864121}
{'neutral': 0.9949496984481812, 'skip': 0.1097157895565033}
{'neutral': 0.4843900501728058, 'skip': 0.0004683129664044827}
{'neutral': 1.0000100135803223, 'skip': 0.004341503139585257}
{'neutral': 0.8311530351638794, 'negative': 0.16886694729328156}
{'neutral': 0.996837317943573, 'skip': 0.04743587225675583}
{'neutral': 1.0000100135803223, 'negative': 0.0029910325538367033}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 1.0000100135803223, 'skip': 0.0004542184469755739}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.6791887283325195, 'negative': 0.08036746829748154}
{'neutral': 0.19194278120994568, 'skip': 0.12253321707248688}
{'neutral': 0.4301573634147644, 'skip': 0.40

{'neutral': 0.5621865391731262, 'positive': 0.05835584178566933}
{'neutral': 0.994455099105835, 'skip': 0.07370626926422119}
{'neutral': 1.0000100135803223, 'negative': 0.01283121295273304}
{'neutral': 0.4843900501728058, 'skip': 0.0004683129664044827}
{'neutral': 0.9931061863899231, 'skip': 0.05501529201865196}
{'neutral': 0.9996055364608765, 'skip': 0.010023568756878376}
{'neutral': 0.9850529432296753, 'skip': 0.9416641592979431}
{'neutral': 0.9679093360900879, 'negative': 0.020974241197109222}
{'neutral': 0.9836069941520691, 'negative': 0.048867784440517426}
{'neutral': 0.9669240117073059, 'negative': 0.03022458776831627}
{'neutral': 0.9715740084648132, 'skip': 0.024433093145489693}
{'neutral': 1.0000100135803223, 'skip': 0.005394937004894018}
{'neutral': 0.9992306232452393, 'skip': 0.016924913972616196}
{'neutral': 1.0000100135803223, 'skip': 0.004478383809328079}
{'neutral': 0.9914325475692749, 'skip': 0.1732981950044632}
{'neutral': 1.0000100135803223, 'negative': 0.0031826822087

{'neutral': 0.9976133108139038, 'skip': 0.01641303114593029}
{'neutral': 0.3775506913661957, 'negative': 0.1294127255678177}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 1.0000100135803223, 'skip': 0.0004542184469755739}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.4532718360424042, 'negative': 0.1067005917429924}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'skip': 0.4688006341457367, 'neutral': 0.008587487041950226}
{'skip': 0.4688006341457367, 'neutral': 0.008587487041950226}
{'neutral': 1.0000100135803223, 'skip': 0.0013770253863185644}
{'neutral': 0.9777238368988037, 'skip': 0.02369946800172329}
{'neutral': 1.0000100135803223, 'skip': 0.0015111821703612804}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.9886782765388489, 'negative': 0.0191340371966362}
{'neutral': 0.9926641583442688, 'skip': 0.07586817443370819}
{'skip': 0.3775506913661957, 'neutral': 

{'neutral': 1.0000100135803223, 'skip': 0.0015111821703612804}
{'neutral': 1.0000100135803223, 'negative': 0.0008930459152907133}
{'neutral': 0.9814634323120117, 'skip': 0.005564924795180559}
{'neutral': 1.0000100135803223, 'negative': 0.0008930459152907133}
{'neutral': 1.0000100135803223, 'positive': 0.0028995804022997618}
{'neutral': 1.0000100135803223, 'skip': 0.010996942408382893}
{'neutral': 0.9995517134666443, 'skip': 0.004765199031680822}
{'neutral': 0.9926641583442688, 'skip': 0.07586817443370819}
{'neutral': 0.960371196269989, 'positive': 0.06755668669939041}
{'neutral': 1.0000100135803223, 'skip': 0.004619571380317211}
{'neutral': 0.9553291201591492, 'positive': 0.04469086229801178}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.9976868629455566, 'skip': 0.002810927340760827}
{'neutral': 0.9525841474533081, 'skip': 0.018556566908955574}
{'neutral': 1.0000100135803223, 'skip': 0.004341503139585257}
{'neutral': 0.9496793746948242, 'skip': 0.04

{'neutral': 0.9982553720474243, 'skip': 0.005921069998294115}
{'neutral': 0.9304682612419128, 'positive': 0.16886694729328156}
{'neutral': 0.8354935646057129, 'negative': 0.1520422399044037}
{'neutral': 0.9942797422409058, 'skip': 0.007587242405861616}
{'neutral': 1.0000100135803223, 'negative': 0.0023331623524427414}
{'neutral': 0.9963027834892273, 'positive': 0.0015587612288072705}
{'neutral': 0.8872146010398865, 'positive': 0.1778208613395691}
{'neutral': 0.7490972280502319, 'skip': 0.4378334879875183}
{'neutral': 1.0000100135803223, 'negative': 0.014513582922518253}
{'neutral': 0.6001983880996704, 'negative': 0.14034625887870789}
{'neutral': 0.9963027834892273, 'skip': 0.02596735954284668}
{'neutral': 0.9820237755775452, 'skip': 0.22271016240119934}
{'neutral': 1.0000100135803223, 'skip': 0.0013770253863185644}
{'neutral': 0.982567548751831, 'skip': 0.08270734548568726}
{'neutral': 0.9996179938316345, 'skip': 0.014073628932237625}
{'neutral': 1.0000100135803223, 'negative': 1.00000

{'positive': 0.22271016240119934, 'speech': 0.09269777685403824}
{'neutral': 1.0000100135803223, 'skip': 0.00048285454977303743}
{'neutral': 1.0000100135803223, 'skip': 0.0015111821703612804}
{'positive': 0.18714269995689392, 'neutral': 0.14034625887870789}
{'neutral': 1.0000100135803223, 'skip': 0.0016078451881185174}
{'neutral': 0.998142421245575, 'negative': 0.03623005375266075}
{'neutral': 1.0000100135803223, 'skip': 0.003717252518981695}
{'neutral': 0.9748311638832092, 'skip': 0.01641303114593029}
{'neutral': 1.0000100135803223, 'negative': 0.0031826822087168694}
{'neutral': 1.0000100135803223, 'negative': 1.0000003385357559e-05}
{'neutral': 0.9991269707679749, 'skip': 0.08510905504226685}
{'neutral': 0.9956784844398499, 'negative': 0.018556566908955574}
{'neutral': 0.9984612464904785, 'negative': 0.010996942408382893}
{'neutral': 1.0000100135803223, 'skip': 0.005230126902461052}
{'positive': 0.9976133108139038, 'skip': 0.008857354521751404}
{'neutral': 0.9886782765388489, 'skip':

KeyboardInterrupt: 